In [16]:
# Imports
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import TextVectorization
import string
import re
from tensorflow.keras import layers

In [17]:
batch_size = 32
raw_train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    "aclImdb/train",
    batch_size=batch_size,
    validation_split=0.2,
    subset="training",
    seed=1337,
)
raw_val_ds = tf.keras.preprocessing.text_dataset_from_directory(
    "aclImdb/train",
    batch_size=batch_size,
    validation_split=0.2,
    subset="validation",
    seed=1337,
)
raw_test_ds = tf.keras.preprocessing.text_dataset_from_directory(
    "aclImdb/test", batch_size=batch_size
)

Found 43952 files belonging to 3 classes.
Using 35162 files for training.
Found 43952 files belonging to 3 classes.
Using 8790 files for validation.
Found 25000 files belonging to 2 classes.


In [18]:
def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, "<br />", " ")
    return tf.strings.regex_replace(
        stripped_html, f"[{re.escape(string.punctuation)}]", ""
    )


max_features = 20000
embedding_dim = 128
sequence_length = 500

vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=max_features,
    output_mode="int",
    output_sequence_length=sequence_length,
)

text_ds = raw_train_ds.map(lambda x, y: x)
vectorize_layer.adapt(text_ds)

def vectorize_text(text, label):
    text = tf.expand_dims(text, -1)
    return vectorize_layer(text), label

train_ds = raw_train_ds.map(vectorize_text)
val_ds = raw_val_ds.map(vectorize_text)
test_ds = raw_test_ds.map(vectorize_text)

train_ds = train_ds.cache().prefetch(buffer_size=10)
val_ds = val_ds.cache().prefetch(buffer_size=10)
test_ds = test_ds.cache().prefetch(buffer_size=10)

In [19]:
inputs = tf.keras.Input(shape=(None,), dtype="int64")

x = layers.Embedding(max_features, embedding_dim)(inputs)
x = layers.Dropout(0.5)(x)

x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
x = layers.GlobalMaxPooling1D()(x)

x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)

predictions = layers.Dense(1, activation="sigmoid", name="predictions")(x)

model = tf.keras.Model(inputs, predictions)

# Compile the model with binary crossentropy loss and an adam optimizer.
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [20]:
epochs = 3

model.fit(train_ds, validation_data=val_ds, epochs=epochs)

Epoch 1/3
1099/1099 [==============================] - 147s 133ms/step - loss: -827461440.0000 - accuracy: 0.2834 - val_loss: -4469441024.0000 - val_accuracy: 0.2882
Epoch 2/3
1099/1099 [==============================] - 112s 102ms/step - loss: -43322408960.0000 - accuracy: 0.2835 - val_loss: -108447416320.0000 - val_accuracy: 0.2882
Epoch 3/3
1099/1099 [==============================] - 112s 102ms/step - loss: -332569214976.0000 - accuracy: 0.2835 - val_loss: -587070701568.0000 - val_accuracy: 0.2882


In [21]:
model.evaluate(test_ds)

782/782 [==============================] - 32s 40ms/step - loss: 2178911174656.0000 - accuracy: 0.5000


[2178911174656.0, 0.5]